# Common usage patterns

## Create a block wavefield modeling operator from the geometry in a JavaSeis/CloudSeis file

```julia
@everywhere using DistributedArrays, DistributedJets, Jets, JetPackWaveFD, TeaSeis, ParallelOperations

function buildblock(ishot,ρ,io)
    h = readframehdrs(io,ishot)
    JopNlProp3DAcoIsoDenQ_DEO2_FDTD(
        sz = -get(prop(io,"SOU_ELEV"), h, 1),
        sy = get(prop(io,"SOU_Y"), h, 1),
        sx = get(prop(io,"SOU_X"), h, 1),
        rz = [-get(prop(io,"REC_ELEV"), h, i) for i = 1:fold(io,h)],
        ry = [get(prop(io,"REC_Y"), h, i) for i = 1:fold(io,h)],
        rx = [get(prop(io,"REC_X"), h, i) for i = 1:fold(io,h)],
        ntrec = size(io,1),
        dtrec = pincs(io,1),
        dtmod = 0.001,
        b = 1 ./ ρ,
        dz = 20.0,
        dy = 20.0,
        dx = 20.0)
end

function buildblocks(I,ρ_futures)
    io = jsopen("data.js")
    ρ = localpart(ρ_futures)
    F = [buildblock(ishot,ρ,io) for ishot in I[1], j in 1:1]
    close(io)
    F
end

io = jsopen("data.js")
nshots = size(io,3) # assume one shot per frame
close(io)

nz,ny,nx=512,512,512
ρ = 1.0*ones(nz,ny,nx)
ρ_futures = bcast(ρ)

F = @blockop DArray(I->buildblocks(I, ρ_futures), (nshots,1))
```

## Populate a distributed block array from a JavaSeis/CloudSeis file

```julia
@everywhere function readblocks!(d)
    io = jsopen("data.js")
    for ishot in localblockindices(d)
        setblock!(d, ishot, readframetrcs(io, ishot))
    end
    close(io)
end

d = zeros(range(F))
@sync for pid in procs(d)
    @async remotecall_fetch(readblocks!, pid, d)
end
```

## Computing cost over a set of shots from a distributed block array

```julia
@everywhere function costperpid(dmod, dobs)
    phi = 0.0
    for iblock in localblockindices(fmod)
        phi += getblock(dobs,iblock) .- getblock(dmod,iblock)
    end
    phi
end

function cost(m, F, dobs)
    dmod = F*m #F is a block operators
    phi = zeros(nprocs(F))
    @sync for (ipid,pid) in enumerate(procs(F))
        @async begin
            phi[ipid] = remotecall_fetch(costperpid, pid, dmod, dobs)
        end
    end
    sum(phi)
end

phi = cost(m,F,dobs)
```
Note that the above can be done in a single line, but the pattern is useful for more interesting cost functions such as optimal transport.  For the above L2 case, the single line would be:
```julia
phi = 0.5*norm(F*m .- d).^2
```